Imports

In [38]:
import os
import json
import requests
from bs4 import BeautifulSoup


1. Send reqest for accessing first webpage with opinious about progect

In [2]:
with open('./cookie.json', 'r', encoding='UTF-8') as jf:
    headers= json.load(jf)

In [40]:
product_id = 107530595
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
while url:
    respomse = requests.get(url, headers=headers)
    if respomse.status_code ==200:
        page_dom = BeautifulSoup(respomse.text, 'html.parser')
        opinions= page_dom.select("div.js_product-review:not(.user-post--highlight)")
        all_opinion=[]
        for opinion in opinions:
            single_opinion = {}
            single_opinion['opinion_id'] = opinion["data-entry-id"]
            single_opinion['author'] = opinion.select_one('span.user-post__author-name').text.strip()
            try:
                single_opinion['recommend'] = opinion.select_one('span.user-post__author-recomendation > em').text.strip()
            except AttributeError:
                single_opinion["recommend"]=None
            single_opinion['stars'] = opinion.select_one('span.user-post__score-count').text.strip()
            single_opinion['content'] = opinion.select_one('div.user-post__text').text.strip()
            single_opinion['pros'] = [tag.text.strip() for tag in opinion.select("div.review-feature__title--positives")]
            single_opinion['cons'] = [tag.text.strip() for tag in opinion.select('div.review-feature__title--negatives')]
            single_opinion['up_votes'] = opinion.select_one('button.vote-yes')['data-total-vote'].strip()
            single_opinion['down_votes'] = opinion.select_one('button.vote-no')['data-total-vote'].strip()
            single_opinion['published'] = opinion.select_one('span.user-post__published > time:nth-child(1)')['datetime'].strip()
            try:
                single_opinion['purchased'] = opinion.select_one('span.user-post__published > time:nth-child(2)')['datetime'].strip()
            except TypeError:
                single_opinion['purchased']=None
            all_opinion.append(single_opinion)
        
        try:
            url = "https://www.ceneo.pl"+ page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            url = None

print(all_opinion)
      

[{'opinion_id': '18882123', 'author': 'r...i', 'recommend': 'Polecam', 'stars': '4,5/5', 'content': 'Jest ok za te pieniądze to super szybko się nagrzewa tylko blacha w drzwiczkach mogła być grubsza ale jest ok', 'pros': [], 'cons': [], 'up_votes': '0', 'down_votes': '0', 'published': '2024-08-14 07:11:10', 'purchased': '2024-08-05 10:10:59'}, {'opinion_id': '19507333', 'author': 'Małgorzata', 'recommend': 'Polecam', 'stars': '5/5', 'content': 'Super cena, wysoka jakość,  grill działa już 4 lata. Teraz kupiłam koledze z pracy.', 'pros': [], 'cons': [], 'up_votes': '0', 'down_votes': '0', 'published': '2025-03-12 19:06:37', 'purchased': '2025-02-20 07:34:18'}, {'opinion_id': '18642236', 'author': 'i...t', 'recommend': 'Polecam', 'stars': '5/5', 'content': 'Produkt dobrej jakości. Jednak zapalarka na dodatkowym palniku nie podpala gazu.', 'pros': [], 'cons': [], 'up_votes': '0', 'down_votes': '0', 'published': '2024-05-13 08:35:58', 'purchased': '2024-04-18 20:22:48'}, {'opinion_id': '17

In [39]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [41]:
with open(f"./opinions{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinion, jf, indent=4, ensure_ascii=False)